## Association Rules - Old vs. New Algorithm

### Dataset: eCommerce_purchase_history_from_jewelry_store

#### 1 Investigating Dataset

#### 1.1 Description
Kaggle Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-jewelry-store <br>
The Dataset is Open-Source <br>
It contains purchase data from December 2018 to December 2021 (3 years) from a medium sized jewelry online store.<br>

In [10]:
import os
import pandas as pd
import glob

In [18]:
# setting the path for joining multiple files
data_raw = T = pd.read_csv("datasets/eCommerce_purchase_history_from_jewelry_store/dataset.csv",sep=",")

data_raw


,event_time,order_id,product_id,quantity,category_id,category_code,brand,price,user_id,gender,colo,metal,gem
0,2018-12-01 11:40:29 UTC,1924719191579951782,1842195256808833386,1,1.806829e+18,jewelry.earring,0.000000e+00,561.51,1.515916e+18,NaN,red,gold,diamond
1,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
2,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.000000e+00,54.66,1.515916e+18,f,white,gold,sapphire
3,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.000000e+00,88.90,1.515916e+18,f,red,gold,diamond
4,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.000000e+00,417.67,1.515916e+18,NaN,red,gold,amethyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95906,2021-12-01 09:47:37 UTC,2719016587653808559,1515966223597820877,1,NaN,0.00,1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN
95907,2021-12-01 09:50:37 UTC,2719018098886705639,1956663830729720074,1,1.806829e+18,NaN,0.000000e+00,229.99,1.515916e+18,NaN,red,gold,NaN
95908,2021-12-01 09:52:06 UTC,2719018846999544322,1956663848219968336,1,1.806829e+18,jewelry.earring,1.000000e+00,630.00,1.515916e+18,f,white,gold,diamond
95909,2021-12-01 09:55:35 UTC,2719020596963836492,1835566854106513435,1,1.806829e+18,jewelry.earring,0.000000e+00,83.42,1.515916e+18,NaN,white,gold,fianit


In [28]:
# Quantity is always 1

data_raw[data_raw["quantity"]==1].index.size == data_raw.index.size

True

In [66]:
# Quantity is always 1

data_raw[data_raw["quantity"]==1].index.size == data_raw.index.size

grouped_mutliple_items = data_raw.groupby(['order_id', 'product_id']).agg({'quantity': ['count']})

#grouped_mutliple_items[grouped_mutliple_items["quantity"]>1]

grouped_mutliple_items.columns = ["quantity"]
grouped_mutliple_items = grouped_mutliple_items.reset_index()
grouped_mutliple_items = grouped_mutliple_items[grouped_mutliple_items["quantity"]>1].sort_values("quantity",ascending=False)

In [72]:
grouped_mutliple_items

,order_id,product_id,quantity
41015,2495379030207889855,1924582891589731058,5
19915,2353608075732059089,1956663840242401751,5
40727,2493738608301703823,1956663831283368958,4
12620,2227042636957483947,1956663846273811079,4
41012,2495371767527244123,1956663831300145187,4
...,...,...,...
22684,2376381688033838048,1956663847725040105,2
22681,2376366506977002204,1515966222682896901,2
22662,2376259120639508790,1956663847725040099,2
22643,2376179809060388868,1956663840225624514,2


In [93]:
# Mutiple items per transaction

import seaborn as sns

mutliple_items_per_transaction = grouped_mutliple_items["quantity"].value_counts()
mutliple_items_per_transaction.rename_axis("frequency")

# lenth_of_transactions

total = mutliple_items_per_transaction.values.sum()

import plotly.express as px
fig = px.bar(mutliple_items_per_transaction, x=mutliple_items_per_transaction.index, y=mutliple_items_per_transaction.values/total,width=400, height=400)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Multiple items per transaction",
    yaxis_tickformat ='0%',
    xaxis_tickformat = ',d',
    yaxis_title="Distribution",
    legend_title="Multiple items per transaction distribution",)

fig.show()


In [94]:
# group by Transaction

import math
import datetime

data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_5936/4105145759.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,date,price,margin
order_id,,,,
1924719191579951782,[1842195256808833386],[2018-12-01],[561.51],[0.1]
1924899396621697920,[1806829193678291446],[2018-12-01],[212.14],[0.1]
1925511016616034733,[1842214461889315556],[2018-12-02],[54.66],[0.1]
1925626951238681511,[1835566849434059453],[2018-12-02],[88.9],[0.1]
1925740842841014667,[1873936840742928865],[2018-12-02],[417.67],[0.1]
...,...,...,...,...
2719016587653808559,[1515966223597820877],[2021-12-01],[nan],[0.1]
2719018098886705639,[1956663830729720074],[2021-12-01],[229.99],[0.1]
2719018846999544322,[1956663848219968336],[2021-12-01],[630.0],[0.1]


In [95]:
# Length of transactions

import seaborn as sns

length_of_transactions = data["product_id"].apply(len).value_counts()
length_of_transactions.rename_axis("frequency")


total = length_of_transactions.sum()

import plotly.express as px
fig = px.bar(length_of_transactions, x=length_of_transactions.index, y=length_of_transactions.values/total)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Number of items per transaction",
    yaxis_tickformat =',.0%',
    yaxis_title="Distribution",
    legend_title="Transaction Length Dsitribution",)

fig.show()

In [19]:
unique_items = data_raw[["product_id","category_id","category_code","brand","price"]].drop_duplicates(subset = "product_id").astype("str")


unique_items

,product_id,category_id,category_code,brand,price
0,1842195256808833386,1.8068292018907384e+18,jewelry.earring,0.0,561.51
1,1806829193678291446,1.8068292018487954e+18,nan,nan,212.14
2,1842214461889315556,1.8068292019159043e+18,jewelry.pendant,1.0,54.66
3,1835566849434059453,1.8068292019159043e+18,jewelry.pendant,0.0,88.9
4,1873936840742928865,1.8068292019242929e+18,jewelry.necklace,0.0,417.67
...,...,...,...,...,...
95833,1515966223700619049,nan,0.00,1.5159156256973775e+18,nan
95838,1515966223419078514,1.8068292018907384e+18,jewelry.earring,1.0,575.21
95867,1515966223714641199,nan,0.00,1.5159156256905946e+18,nan
95870,1515966223714641225,nan,0.00,1.5159156256905946e+18,nan


In [114]:
unique_items["price"].astype("float").median()

261.66999999999996

In [20]:
data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_5936/16707732.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))


,product_id,date,price,margin
order_id,,,,
1924719191579951782,[1842195256808833386],[2018-12-01],[561.51],[0.1]
1924899396621697920,[1806829193678291446],[2018-12-01],[212.14],[0.1]
1925511016616034733,[1842214461889315556],[2018-12-02],[54.66],[0.1]
1925626951238681511,[1835566849434059453],[2018-12-02],[88.9],[0.1]
1925740842841014667,[1873936840742928865],[2018-12-02],[417.67],[0.1]
...,...,...,...,...
2719016587653808559,[1515966223597820877],[2021-12-01],[nan],[0.1]
2719018098886705639,[1956663830729720074],[2021-12-01],[229.99],[0.1]
2719018846999544322,[1956663848219968336],[2021-12-01],[630.0],[0.1]


In [35]:
import seaborn as sns

length_of_transactions = data["product_id"].apply(len).value_counts()
length_of_transactions.rename_axis("frequency")

# lenth_of_transactions

total = length_of_transactions.sum()

import plotly.express as px
fig = px.bar(length_of_transactions, x=length_of_transactions.index, y=length_of_transactions.values/total)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Number of items per transaction",
    yaxis_tickformat =',.0%',
    yaxis_title="Distribution",
    legend_title="Transaction Length Dsitribution",)

fig.show()


c:\ProgramData\Anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
c:\ProgramData\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
c:\ProgramData\Anaconda3\lib\site-packages\xarray\core\pycompat.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion(duck_array_module.__version__)
c:\ProgramData\Anaconda3\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
c:\ProgramData\Anaconda3\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version

In [96]:
length_of_transactions.sum()

74760

#### 3.1.1 Classical Association Rules

In [109]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.004,
    maxSupRatio=1,
    minConf=0,
    item_col=1
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[],NA,[1515966223577083202],468,[1515966223577083202],468,0.006260,NA,NA,NA
1,[],NA,[1956663847725040099],362,[1956663847725040099],362,0.004842,NA,NA,NA
2,[],NA,[1956663840242401751],301,[1956663840242401751],301,0.004026,NA,NA,NA


In [108]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.


rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled


,rules_index,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,0,1515966223577083202,468,1515966223577083202,1.80682920194107e+18,jewelry.brooch,0.0,10.27
1,1,1956663847725040099,362,1956663847725040099,1.8068292018823498e+18,jewelry.ring,0.0,200.45
2,2,1956663840242401751,301,1956663840242401751,1.8068292018823498e+18,jewelry.ring,0.0,259.97
3,3,1956663840309510725,274,1956663840309510725,1.8068292018823498e+18,jewelry.ring,0.0,133.55
4,4,1956663830872326617,266,1956663830872326617,1.8068292018487954e+18,nan,0.0,140.62
5,5,1956663847666319760,266,1956663847666319760,1.8068292018823498e+18,jewelry.ring,0.0,188.7
6,6,1352907200745439279,265,1352907200745439279,1.8068292018823498e+18,jewelry.ring,0.0,215.14
7,7,1956663836207481430,259,1956663836207481430,1.8068292018823498e+18,jewelry.ring,0.0,119.77
8,8,1956663845787271453,258,1956663845787271453,1.8068292018487954e+18,nan,0.0,445.95
9,9,1956663831283368958,253,1956663831283368958,1.8068292018823498e+18,jewelry.ring,0.0,242.9


#### 3.1.2 Classical Association Rules (Limitations on transaction length as at least 2)

In [120]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0003,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    minTransactionLength=2,
    maxTransactionLength=10
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[1944945390285488528],221,[1313614230015967859],170,"[1944945390285488528, 1313614230015967859]",51,0.000682,0.230769,101.484163,0.228495
1,[1944422271028298685],154,[1937902773722939556],131,"[1944422271028298685, 1937902773722939556]",34,0.000455,0.220779,125.995836,0.219027
2,[1956663836207481430],259,[1956663836207481431],183,"[1956663836207481431, 1956663836207481430]",29,0.000388,0.111969,45.742136,0.109521
3,[1313614230015967859],170,[1343446704099164925],90,"[1343446704099164925, 1313614230015967859]",26,0.000348,0.152941,127.043137,0.151737
4,[1956663840309510725],274,[1956663836207481430],259,"[1956663840309510725, 1956663836207481430]",25,0.000334,0.091241,26.336556,0.087776
5,[1956663848287077291],102,[1956663848287077290],100,"[1956663848287077291, 1956663848287077290]",24,0.000321,0.235294,175.905882,0.233957
6,[1956663840242401751],301,[1956663846349308653],169,"[1956663840242401751, 1956663846349308653]",23,0.000308,0.076412,33.802119,0.074151
7,[1956663847666319760],266,[1352907200745439279],265,"[1956663847666319760, 1352907200745439279]",23,0.000308,0.086466,24.393247,0.082921


In [123]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled


,rules_index,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,0,1944945390285488528,51,1944945390285488528,1.8068292018823498e+18,jewelry.ring,1.0,534.11
1,0,1313614230015967859,51,1313614230015967859,1.8068292018907384e+18,jewelry.earring,1.0,903.97
2,1,1944422271028298685,34,1944422271028298685,1.8068292018823498e+18,jewelry.ring,1.0,493.01
3,1,1937902773722939556,34,1937902773722939556,1.8068292018907384e+18,jewelry.earring,1.0,856.03
4,2,1956663836207481431,29,1956663836207481431,1.8068292018823498e+18,jewelry.ring,0.0,145.62
5,2,1956663836207481430,29,1956663836207481430,1.8068292018823498e+18,jewelry.ring,0.0,119.77
6,3,1313614230015967859,26,1313614230015967859,1.8068292018907384e+18,jewelry.earring,1.0,903.97
7,3,1343446704099164925,26,1343446704099164925,1.8068292019159043e+18,jewelry.pendant,1.0,410.82
8,4,1956663836207481430,25,1956663836207481430,1.8068292018823498e+18,jewelry.ring,0.0,119.77
9,4,1956663840309510725,25,1956663840309510725,1.8068292018823498e+18,jewelry.ring,0.0,133.55


#### 3.2 Profit based Association Rules (No limitations on transaction length)

In [131]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"
rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.002,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules


,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[],NA,[1806829191514031042],88,[1806829191514031042],88,0.001241,NA,NA,NA,247480.520000,0.007545,0.0,0.000000,247480.52,0.000000
1,[],NA,[1956663836668854462],95,[1956663836668854462],95,0.001339,NA,NA,NA,219755.440000,0.006699,0.0,0.000000,219755.44,0.000000
2,[],NA,[1956663836668854461],159,[1956663836668854461],159,0.002242,NA,NA,NA,201361.600000,0.006139,0.0,0.000000,201361.60,0.000000
3,[],NA,[1956663847708262860],107,[1956663847708262860],107,0.001508,NA,NA,NA,184792.210000,0.005634,0.0,0.000000,184792.21,0.000000
4,[],NA,[1313614230015967859],170,[1313614230015967859],170,0.002397,NA,NA,NA,153674.900000,0.004685,0.0,0.000000,153674.90,0.000000
5,[],NA,[1956663831425974504],111,[1956663831425974504],111,0.001565,NA,NA,NA,153430.270000,0.004677,0.0,0.000000,153430.27,0.000000
6,[],NA,[1956663831375642763],129,[1956663831375642763],129,0.001819,NA,NA,NA,135714.530000,0.004137,0.0,0.000000,135714.53,0.000000
7,[],NA,[1515966223500113689],5,[1515966223500113689],5,0.000070,NA,NA,NA,132122.600000,0.004028,0.0,0.000000,132122.60,0.000000
8,[],NA,[1944945390285488528],221,[1944945390285488528],221,0.003116,NA,NA,NA,122845.300000,0.003745,0.0,0.000000,122845.30,0.000000
9,[],NA,[1956663846449972073],146,[1956663846449972073],146,0.002058,NA,NA,NA,121057.360000,0.003691,0.0,0.000000,121057.36,0.000000


In [132]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled


,rules_index,antecedent&consequent,sup_ant&cons,product_id,category_id,category_code,brand,price
0,0,1806829191514031042,88,1806829191514031042,1.8068292018487954e+18,nan,0.0,2780.68
1,1,1956663836668854462,95,1956663836668854462,1.8068292018487954e+18,nan,0.0,2265.52
2,2,1956663836668854461,159,1956663836668854461,1.8068292018487954e+18,nan,0.0,1258.51
3,3,1956663847708262860,107,1956663847708262860,1.8068292018487954e+18,nan,0.0,1727.03
4,4,1313614230015967859,170,1313614230015967859,1.8068292018907384e+18,jewelry.earring,1.0,903.97
5,5,1956663831425974504,111,1956663831425974504,1.8068292018487954e+18,nan,0.0,1357.79
6,6,1956663831375642763,129,1956663831375642763,1.8068292018823498e+18,jewelry.ring,0.0,1020.41
7,7,1515966223500113689,5,1515966223500113689,1.8068292018823498e+18,jewelry.ring,0.0,26424.52
8,8,1944945390285488528,221,1944945390285488528,1.8068292018823498e+18,jewelry.ring,1.0,534.11
9,9,1956663846449972073,146,1956663846449972073,1.8068292018487954e+18,nan,0.0,829.16


#### 3.2 Profit based Association Rules (Limitations on transaction length as at least 2)

In [137]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"
rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0009,
    maxSupRatio=1,
    minTransactionLength=2,
    maxTransactionLength=10,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[1944945390285488528],221,[1313614230015967859],170,"[1944945390285488528, 1313614230015967859]",51,0.000719,0.230769,96.289140,0.228373,74451.385385,0.002270,122845.30,-48393.914615,46102.470000,-94496.384615
1,[1944422271028298685],154,[1937902773722939556],131,"[1944422271028298685, 1937902773722939556]",34,0.000479,0.220779,119.546049,0.218932,46846.977273,0.001428,80360.63,-33513.652727,29105.020000,-62618.672727
2,[1956663846340920038],43,[1956663845602721816],34,"[1956663846340920038, 1956663845602721816]",16,0.000226,0.372093,776.284542,0.371614,36891.263256,0.001125,47610.20,-10718.936744,19175.840000,-29894.776744
3,[1860421116053422140],73,[1845870160691331313],58,"[1860421116053422140, 1845870160691331313]",18,0.000254,0.246575,301.557393,0.245758,36455.853699,0.001111,54359.10,-17903.246301,23052.240000,-40955.486301
4,[1956663836668854462],95,[1806829191514031042],88,"[1806829191514031042, 1956663836668854462]",7,0.000099,0.073684,59.393660,0.072444,35878.456560,0.001094,219755.44,-183876.983440,19685.950455,-203562.933895
5,[1313614230015967859],170,[1343446704099164925],90,"[1343446704099164925, 1313614230015967859]",26,0.000367,0.152941,120.539739,0.151672,34184.540000,0.001042,153674.90,-119490.360000,10681.320000,-130171.680000
6,[1956663845736939738],74,[1956663836392031040],55,"[1956663845736939738, 1956663836392031040]",17,0.000240,0.229730,296.280344,0.228954,32443.744189,0.000989,45441.59,-12997.845811,22004.460000,-35002.305811
7,[1956663845812437315],36,[1956663845778882836],17,"[1956663845812437315, 1956663845778882836]",7,0.000099,0.194444,811.325163,0.194205,30057.025833,0.000916,63351.03,-33294.004167,17738.770000,-51032.774167


In [139]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.


rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons","profit_associated","perc_of_total_profit"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)

rules_labeled =pd.merge(rules_exploded,unique_items, how="inner", left_on="antecedent&consequent",right_on="product_id").sort_values("rules_index").reset_index(drop=True)

rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled["profit_associated"] = rules_labeled["profit_associated"].astype(int)

rules_labeled.head(16)

,rules_index,antecedent&consequent,sup_ant&cons,profit_associated,perc_of_total_profit,product_id,category_id,category_code,brand,price
0,0,1944945390285488528,51,74451,0.002270,1944945390285488528,1.8068292018823498e+18,jewelry.ring,1.0,534.11
1,0,1313614230015967859,51,74451,0.002270,1313614230015967859,1.8068292018907384e+18,jewelry.earring,1.0,903.97
2,1,1944422271028298685,34,46846,0.001428,1944422271028298685,1.8068292018823498e+18,jewelry.ring,1.0,493.01
3,1,1937902773722939556,34,46846,0.001428,1937902773722939556,1.8068292018907384e+18,jewelry.earring,1.0,856.03
4,2,1956663846340920038,16,36891,0.001125,1956663846340920038,1.8068292018823498e+18,jewelry.ring,0.0,1082.05
5,2,1956663845602721816,16,36891,0.001125,1956663845602721816,1.8068292018907384e+18,jewelry.earring,0.0,1198.49
6,3,1860421116053422140,18,36455,0.001111,1860421116053422140,1.8068292018823498e+18,jewelry.ring,0.0,671.1
7,3,1845870160691331313,18,36455,0.001111,1845870160691331313,1.8068292018907384e+18,jewelry.earring,0.0,1280.68
8,4,1806829191514031042,7,35878,0.001094,1806829191514031042,1.8068292018487954e+18,nan,0.0,2780.68
9,4,1956663836668854462,7,35878,0.001094,1956663836668854462,1.8068292018487954e+18,nan,0.0,2265.52


#### 3.3 Sensitivity Analysis

In [ ]:
import math
import datetime

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    df_data_simple_withprofit,
    minSupRatio=0.03,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=2,
    max_profit = 100,
    profit_sensitivity = lambda x : 1 * x
    ) #Only Date

rules


NameError: name 'df_data_simple_withprofit' is not defined